<a href="https://colab.research.google.com/github/akimotolab/CMAES_Tutorial/blob/main/a1_minmax_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evolution Strategyによるミニマックス最適化

ここではESを用いたミニマックス最適化について紹介する．
特に，
1. 局所探索を志向した方法である Adversarial-CMA-ES [1] （pythonコード： https://gist.github.com/youheiakimoto/ab51e88c73baf68effd95b750100aad0#comments）

2. 大域探索＆並列化を志向した方法である WRA-CMA-ES [2] (pythonコード： https://github.com/akimotolab/worstcase-ranking-approximation)

を紹介する．

[1] Youhei Akimoto, Yoshiki Miyauchi, and Atsuo Maki. 2022.
Saddle Point Optimization with Approximate Minimization Oracle and Its Application to Robust Berthing Control.
ACM Trans. Evol. Learn. Optim. 2, 1, Article 2 (March 2022), 32 pages.
DOI:https://doi.org/10.1145/3510425

[2] Atsuhiro Miyagi, Yoshiki Miyauchi, Atsuo Maki, Kazuto Fukuchi, Jun Sakuma, and Youhei Akimoto. 2023. Covariance Matrix Adaptation Evolutionary Strategy with Worst-Case Ranking Approximation for Min-Max Optimization and Its Application to Berthing Control Tasks. ACM Trans. Evol. Learn. Optim. 3, 2, Article 8 (June 2023), 32 pages. https://doi.org/10.1145/3603716

## 背景

ESが活用される場面では，多くの場合，目的関数$f$は与えられた入力$x$に対する数値シミュレーションを介して計算される．
当然，シミュレーションによって計算される目的関数値が，現実世界に実装した場合の解$x$の良さを表していることを期待するが，現実的にはそうならない場合が多く見られる．
例えば，タイヤの形状を最適化するような場面を考える．どのような指標を良さに表すのかについては議論しないが，タイヤの形状の良し悪しを評価するためにシミュレーションを実施すると言っても，路面の状態や気象状況によってシミュレーションの結果が変わることは容易に想像される．
ここでいう，路面の状態や気象状況などの，シミュレーション実施時には知り得ない（運用時の天候はシミュレーション実施時にはわからない），もしくはそもそも一意に定まらない（天候も路面状態も運用する時々で変化する）ような要因を不確実性などと呼ぶ．
これを$y$で表すことにする．
この$y$は，シミュレーションの条件設定を表していると考えることもできる．
一般的な最適化の場合，最適化実施前に$y$を予め決めておき，
$$
x^* = \mathrm{argmax}_x f(x, y)
$$
となるような解$x^*$を求めようとしている，と考えられる．
しかし，最適化実施時に設定した$y_\mathrm{sim}$と運用時に直面する$y$が異なることで，目的関数値が著しく悪化する，すなわち$f(x^*, y_\mathrm{sim}) \ll f(x^*, y)$となる可能性がある．

### テスト問題での例
具体的なテスト問題２つでこの現象を確認する．

#### 凹凸二次関数
１つ目の問題は凹凸二次関数
$$
f(x, y) = a x^2 + b x y - c y^2
$$
である．
簡単のため，定義域は$\mathbb{R} \times \mathbb{R}$とする．
係数については，$a, c > 0$，$b \in \mathbb{R}$とする．
各$y$について，最適な$x$は
$$
x^*(y) = \frac{-by}{2a}
$$
であることから，最適な$x$が$y$に依存することがわかる．
とりわけ，$| b | / a$が大きいほど，$y$の変化に対して最適解が大きく変化することがわかる．
各$y$について，最適解をとった場合の目的関数値は
$$
f(x^*(y), y) = - \left( \frac{b^2}{4a} + c\right) y^2
$$
となる．
しかし，最適化時に用いた$y_\mathrm{sim}$と運用時の$y$が異なれば，
$$
f(x^*(y_\mathrm{sim}), y) = \frac{b^2}{4 a} \left( y_\mathrm{sim} - y\right)^2 - \left(\frac{b^2}{4 a} + c\right) y^2
$$
となる．
すなわち，第一項の分だけ，本来の最適な値よりも損をすることになる．
第一項はシミュレーションに用いた$y_\mathrm{sim}$と運用時の$y$が離れているほどに大きくなることも見て取れる．

#### 双線形関数
２つ目の問題は双線形関数
$$
f(x, y) = x y
$$
である．
定義域は$[-1, 1]\times [-1, 1]$とする．
各$y$について最適な解は
$$
x^*(y) = -\mathrm{sign}(y)
$$
となる．
$y=0$の場合には，任意の$x$が最適となるが，簡単のため$x^* = 0$と考えることにする．
この問題の場合，最適解が不連続に変化することになる．
各$y$について，最適解をとった場合の目的関数値は
$$
f(x^*(y), y) = - |y|
$$
となる．
他方，最適化時に用いた$y_\mathrm{sim}$と運用時の$y$が異なれば，
$$
f(x^*(y_\mathrm{sim}), y) = - \mathrm{sign}(y_\mathrm{sim}) y
$$
となる．
最適化時に用いた$y_\mathrm{sim}$と運用時の$y$の符号が一致している限り，最適化で得られた解が運用時にも最適な解となるが，符号が一致していない場合には性能劣化が起こることが見て取れる．

## ミニマックス最適化問題
不確実性を含む問題においては，様々なアプローチが考えられるが，ここでは「考えうる最悪なシナリオでの性能を最適化する解を求める問題」を考える．
運用時に起こり得る不確実性の集合$Y$を予め特定できると考える．
ある解$x$が与えられたもとで，この$x$にとって最悪性能を与える$y \in Y$を考える．
この最悪性能を表す指標を
$$
F_Y(x) = \max_{y \in Y} f(x, y)
$$
と書けば，この最悪性能$F_Y$を最小化するような解を見つけることがここでの目的であり，これはミニマックス最適化
$$
x^* = \mathrm{argmax}_x F_Y(x) = \mathrm{argmax}_x \max_{y \in Y} f(x, y)
$$
として定式化される．
ESが対象とするような最適化問題において，目的関数はブラックボックスであるため，最悪性能$F_Y(x)$を解析的に求めることはできない．そのため，$F_Y(x)$を求めるために，各$x$に対して最悪なシナリオを求める必要がある．実用上は$Y$を適切に定めることも困難さの一つといえるが，この資料ではその後の最適化のみに着目する．